In [ ]:
%pip install llama-index
%pip install huggingface_hub

In [ ]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface

In [ ]:
%pip install llama_index
%pip install llama-index-llms-huggingface


In [ ]:
!pip install llama-index-vector-stores-faiss faiss-cpu llama-index-node-store

ERROR: Could not find a version that satisfies the requirement llama-index-node-store (from versions: none)
ERROR: No matching distribution found for llama-index-node-store


In [ ]:
from google.colab import userdata

In [ ]:
import openai
openai.api_key=userdata.get("OPENAI_API_KEY")

In [ ]:
#print(openai.api_key)

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM

In [ ]:
import os

In [ ]:
from huggingface_hub import login
login(token=os.getenv("HF_TOKEN"))

In [ ]:
llm = HuggingFaceLLM(
    model_name="google/gemma-1.1-2b-it",
    tokenizer_name = 'google/gemma-1.1-2b-it' # Use your model name here # Your Hugging Face API key
    #context_window=8192,                 # Optional: context window for your model
    #max_new_tokens=256,                  # Optional: generation params                    # Optional: generation params
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [ ]:
from datasets import load_dataset
# Load a Hugging Face dataset (example: 'ag_news')
dataset = load_dataset("ag_news", split="train[:5]")

In [ ]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 5
})

In [ ]:
from llama_index.core import Document

In [ ]:
docs = [Document(text=item['text']) for item in dataset]

In [ ]:
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore

EMBED_DIM=384
# 1. Create a FAISS index (L2 for most cases)
faiss_index = faiss.IndexFlatL2(EMBED_DIM)

# 2. Pass it to FaissVectorStore
vector_store = FaissVectorStore(faiss_index=faiss_index)

In [ ]:
#vector_store

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=200,
    separator=" "
    #separators=[" ","\\",","],
)

In [ ]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(docs):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [ ]:
docs[0]

Document(id_='b33f38e3-aa08-4194-a66e-aaa63646d0aa', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}')

In [ ]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = docs[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [ ]:
nodes[0]

TextNode(id_='c6f4a1d9-8548-4050-a753-1f1f6c707770', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}')

In [ ]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [ ]:
nodes[0]

TextNode(id_='c6f4a1d9-8548-4050-a753-1f1f6c707770', embedding=[-0.0318681076169014, -0.043487221002578735, -0.03360360488295555, 0.01549158338457346, 0.036888737231492996, 0.019334979355335236, 0.006151259411126375, 0.013599374331533909, 0.047318313270807266, -0.011293723247945309, 0.014934460632503033, -0.00021026594913564622, -0.013186085037887096, 0.04613563045859337, 0.027558837085962296, -0.01784738525748253, 0.04003654047846794, -0.031517431139945984, -0.051589660346508026, 0.05837508663535118, -0.03136278688907623, -0.0496353954076767, 0.025241119787096977, -0.0366138331592083, 0.04002584517002106, -0.017845449969172478, -0.007586346939206123, -0.002500808099284768, -0.04905818775296211, -0.17541104555130005, -0.020033732056617737, -0.03356899693608284, 0.008014210499823093, -0.018595481291413307, 0.04147814214229584, -0.01160959992557764, -0.009828003123402596, 0.037318434566259384, 0.02712881565093994, 0.03972791135311127, -0.0050964574329555035, 0.0067004794254899025, -0.016

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.indices.vector_store import VectorStoreIndex
index = VectorStoreIndex(nodes)
retriever = index.as_retriever()

In [ ]:
results = retriever.retrieve("What is the latest news about crude oil?")
for node in results:
    print(node.get_content())

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}